[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb)

📚 [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook)

# Intro to LangChain

LangChain is a popular framework that allow users to quickly build apps and pipelines around **L**arge **L**anguage **M**odels. It can be used to for chatbots, **G**enerative **Q**uestion-**A**nwering (GQA), summarization, and much more.

<span style="color: #705a8e;">
    The core idea of the library is that we can <b>_chain_</b> together different components to create more advanced use-cases around LLMs. Chains may consist of multiple components from several modules:
</span>

* **Prompt templates**: These are predefined structures or formats that guide the generation of prompts for different applications. They can be designed to suit various use-cases such as chatbots, question-answering systems etc. For example, a template for an ELI5 (Explain Like I'm Five) question-answering system might include a simple, easy-to-understand question followed by a more detailed explanation.

* **LLMs**: LLMs are AI models trained on a large corpus of text data. They have the ability to generate human-like text based on the input or prompt they are given. Examples of LLMs include models like GPT-3 and BLOOM. These models are capable of understanding and generating text for a wide range of tasks, from writing essays to answering questions and even coding.

* **Agents**: In the context of LangChain, agents are entities that use LLMs to decide which actions to take based on the input they receive. Agents can utilize various tools, such as web search engines or calculators, to perform tasks. The tasks and their sequence are defined in a logical loop of operations, which can include steps like receiving input, processing it, choosing an action, and providing an output.

* **Memory**: In LangChain, memory refers to the system's ability to **store and recall information**. There are two types of memory: short-term and long-term. Short-term memory refers to information that is kept available for <span style="color: skyblue;"><b>immediate use</b></span>, for instance, the context of a conversation in a chatbot. Long-term memory, on the other hand, is information that the system retains over time, like <span style="color: skyblue;"><b>facts or knowledge</b></span> that the system has learned or been trained on.

In [ ]:
!pip install -qU langchain

# Using LLMs in LangChain

🔗 LangChain supports several LLM providers, like Hugging Face and OpenAI.

🔍 Let's start our exploration of LangChain by learning how to use a few of these different LLM integrations.

## 🤗 Hugging Face

We first need to install additional prerequisite libraries:

In [ ]:
!pip install -qU huggingface_hub

For Hugging Face models we need a Hugging Face Hub API token. We can find this by first getting an account at [HuggingFace.co](https://huggingface.co/) and clicking on our profile in the top-right corner > click *Settings* > click *Access Tokens* > click *New Token* > set *Role* to *write* > *Generate* > copy and paste the token below:

In [ ]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'HF_API_KEY'

We can then generate text using a HF Hub model (we'll use `google/flan-t5-x1`) using the Inference API built into Hugging Face Hub.

_(The default Inference API doesn't use specialized hardware and so can be slow and cannot run larger models like `bigscience/bloom-560m` or `google/flan-t5-xxl`)_

📝 Notes: If using Jupyter Notebook, you might an error on import which you can resolve using `pip install pydantic==1.8`

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

# initialize HF LLM
flan_t5 = HuggingFaceHub(
    repo_id="google/flan-t5-base", # repo_id="google/flan-t5-xl" XL is not availble for free tier users
    model_kwargs={"temperature":1e-10}
)

# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt,
    llm=flan_t5
)

question = "Which NFL team won the Super Bowl in the 2010 season?"

# print(llm_chain.run(question))
print(llm_chain.invoke(question))

If we'd like to ask multiple questions we can by passing a list of dictionary objects, where the dictionaries must contain the input variable set in our prompt template (`"question"`) that is mapped to the question we'd like to ask.

In [ ]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
res = llm_chain.generate(qs)
res

It is a LLM, so we can try feeding in all questions at once:

In [ ]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=flan_t5
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.invoke(qs_str))

But with this model it doesn't work too well, we'll see this approach works better with different models soon.

## OpenAI

Start by installing additional prerequisites:

In [ ]:
!pip install -qU openai

We can also use OpenAI's generative models. The process is similar, we need to
give our API key which can be retrieved by signing up for an account on the
[OpenAI website](https://openai.com/api/) (see top-right of page). We then pass the API key below:

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

If using OpenAI via Azure you should also set:

```python
os.environ['OPENAI_API_TYPE'] = 'azure'
# API version to use (Azure has several)
os.environ['OPENAI_API_VERSION'] = '2022-12-01'
# base URL for your Azure OpenAI resource
os.environ['OPENAI_API_BASE'] = 'https://your-resource-name.openai.azure.com'
```

Then we decide on which model we'd like to use, there are several options but we will go with `gpt-3.5-turbo-instruct`:

In [ ]:
from langchain import OpenAI

davinci = OpenAI(model_name='gpt-3.5-turbo-instruct')

Alternatively if using Azure OpenAI we do:

```python
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name="your-azure-deployment", 
    model_name="text-davinci-003"
)
```

We'll use the same simple question-answer prompt template as before with the Hugging Face example. The only change is that we now pass our OpenAI LLM `davinci`:

In [ ]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

print(llm_chain.run(question))

The same works again for multiple questions using `generate`:

In [ ]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
llm_chain.generate(qs)

Note that the below format doesn't feed the questions in iteratively but instead all in one chunk.

In [ ]:
qs = [
    "Which NFL team won the Super Bowl in the 2010 season?",
    "If I am 6 ft 4 inches, how tall am I in centimeters?",
    "Who was the 12th person on the moon?",
    "How many eyes does a blade of grass have?"
]
print(llm_chain.run(qs))

Now we can try to answer all question in one go, as mentioned, more powerful LLMs like `text-davinci-003` will be more likely to handle these more complex queries.

In [ ]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

---